In [1]:
from astropy.io import fits
import re, yaml
from BAT_data_processing import BAT_data

In [2]:
path2soft = '/home/anastasia/work/Cross-calibration/'
path2BATsoft = '/home/anastasia/work/Cross-calibration/BAT_software/'
path2BATdata = '/mnt/win/work/Cross_calibration/BAT_data/'


'''Getting the BAT burst info and extracting mission elapsed time'''

MET = dict()
long_trig = dict()

with open(f'{path2BATsoft}/BAT_summary_general.txt', 'r') as f:
    next(f)
    for line in f:
        GRBname, Trig_ID, Trig_time_met, Trig_time_UTC, RA_ground, DEC_ground, Image_position_err, Image_SNR, \
        T90, T90_err, T50, T50_err, Evt_start_sincetrig, Evt_stop_sincetrig, pcode, Trigger_method, \
        XRT_detection, comment = re.split(r'\s+\|\s+', line)
        if re.search('\D+', Trig_ID): continue

        GRBname = GRBname.replace('GRB', '')

        if len(Trig_ID) == 6: Trig_ID += '000'
        elif len(Trig_ID) < 6: print('GRB', GRBname, Trig_ID, 'is too short!')

        long_trig[GRBname] = '{:011d}'.format(int(Trig_ID))
        MET[GRBname] = Trig_time_met
        
        
'''Extracting difference in trigger times'''

dT0 = dict()
file = f'{path2soft}delta_trig_time.txt'

with open(file, 'r') as f:
    next(f)
    for line in f:
        ID, Name, dt_BAT, dt_GBM = line.split()
        dT0[Name] = float(dt_BAT)

        
'''Extracting light propagation time'''

ToF = dict()

with open(f'{path2soft}KW_software/light_propagation_times.dat') as f:
    next(f)
    for line in f:
        ID, GRBname, swift, fermi = line.split()
        ToF[GRBname] = float(swift)

        

In [3]:
'''Extracting Swift slew times'''

slew_time_dict = dict()

with open(path2BATsoft+'slew_times.txt', 'w') as outfile:
    with open(path2soft+'List_of_spectra_for_fitting.txt') as f:
        next(f)
        print('ID', 'GRB', 'PreSlew', 'InSlew', 'PostSlew', file=outfile)

        for line in f:
            if re.match('#', line): continue
            ID, name, spectra, bg_spectra = line.split('\t')

            print(ID, name, end=' ', file=outfile)

            slew_time_list = list()
            
            for suffix in 'preslew', 'inslew', 'postslew':
                fits_filename = path2BATdata + f'{name}/{long_trig[name]}-results/gti/sw{long_trig[name]}b_grb_{suffix}.gti'
                
                with fits.open(fits_filename) as hdul:
                    ti = hdul[1].data[0][0] - hdul[0].header['TRIGTIME'] - dT0[name] - ToF[name]
                    tf = hdul[1].data[0][1] - hdul[0].header['TRIGTIME'] - dT0[name] - ToF[name]
                    slew_time_list.append((ti, tf))
                                        
                    print(f'{ti:.3f}', f'{tf:.3f}', end=' ', file=outfile)
                    
            print(file=outfile)

            slew_time_dict.update({name: slew_time_list})


In [4]:
# name = '180402A' # short GRB 
# ID = '4630'
spectra = dict()
with open(path2soft+'KW_software/acc_times.yml', 'r') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
    
    for entry in data:
        spectra[entry['GCN_name']] = list()

        for spec in entry['fit_sp']:
            spectra[entry['GCN_name']].append(spec)
        
# spectra

In [5]:
name = '180402A'

test_data = BAT_data(GRB_name=name, trig_ID=long_trig[name], dT0=dT0[name], MET=MET[name], ToF=ToF[name], \
                     slew_interval=list(slew_time_dict[name][1]), sp_ID=list(spectra[name][0])[0], \
                     sp_interval=list(spectra[name][0].values())[0], path2BATdata=path2BATdata, \
                     new_folder='new_spectra', verbose=True)

Burst name: 180402A
Spectrum: 1_4 lasts from 0.0 s to 0.256 s
Long BAT trigger ID: 00821103000
Swift Mission Elapsed Time: 544355120.0 s
Time of light propagation between KW and Swift: -0.956 s
Difference in KW and BAT trigger times: 1.074 s
Time interval when the spacecraft slews: from 9.583 s to 63.383 s
Path to the BAT data folder: /mnt/win/work/Cross_calibration/BAT_data/
Folder where the new BAT fits-files will be collected: new_spectra


In [6]:
test_data.make_rsp()


        Calculating DRM weights and running a slew-corected DRM
        
Time intervals for aux DRMs: [0.0, 0.256]
Weights of the aux DRMs: 1.0

        Making a slew-corrected DRM
        
batbinevt infile=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/events/sw00821103000b_all.evt outfile=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/new_spectra/sw00821103000b_1_aux.pha outtype=PHA timedel=0 timebinalg=u             energybins=CALDB:80 tstart=544355120.118 tstop=544355120.3740001 outunits=RATE detmask=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/auxil/sw00821103000b_qmap.fits             clobber=yes ecol=ENERGY weighted=YES
batupdatephakw infile=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/new_spectra/sw00821103000b_1_aux.pha auxfile=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/auxil/sw00821103000b_all.evaux
batdrmgen infile=/mnt/win/work/Cross_calibration/BAT_da

In [7]:
test_data.make_pha()


        Making a pha file
        
batbinevt infile=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/events/sw00821103000b_all.evt outfile=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/new_spectra/BAT_00821103000_sp1_4.pha outtype=PHA timedel=0.0 timebinalg=u         tstart=544355120.118 tstop=544355120.3740001 energybins=CALDB:80 outunits=RATE detmask=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/auxil/sw00821103000b_qmap.fits clobber=YES         ecol=ENERGY weighted=YES
batupdatephakw infile=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/new_spectra/BAT_00821103000_sp1_4.pha auxfile=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/auxil/sw00821103000b_all.evaux
batphasyserr infile=/mnt/win/work/Cross_calibration/BAT_data//180402A/00821103000-results/new_spectra/BAT_00821103000_sp1_4.pha syserrfile=CALDB
